In [2]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.cohort import cohort
from utils.ethogram import ethogram

Autosaving every 180 seconds


In [3]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
# fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'

#
cohort = cohort.CohortProcessor(fname_spreadsheet)
#cohort.fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
cohort.load_database()

#
cohort.ethogram = ethogram

# print (list(zip(cohort.PDays,cohort.Start_times))[0])
# print (list(zip(cohort.PDays,cohort.Start_times))[1])
# print (list(zip(cohort.PDays,cohort.Start_times))[100])

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename Start time
0           0       2     Day  2020_07_20_09_53_59_813912_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_09_53_59_813912_compressed_Day.slp   09:53:59
1           1       2     Day  2020_07_20_10_16_08_629297_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_16_08_629297_compressed_Day.slp   10:16:08
2           2       2     Day  2020_07_20_10_38_20_043555_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_38_20_043555_compressed_Day.slp   10:38:20
3           3       2     Day  2020_07_20_11_00_31_566455_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_11_00_31_566455_compressed_Day.slp   11:00:31
4           4       

In [29]:
######################################################
############# PREPROCESS FEATURE TRACKS ##############
######################################################

# NOTE The hybrid video processing seems to break when using parallel flag

#
cohort.n_cores = 30
cohort.parallel = False
cohort.interpolate_flag = False
cohort.exclude_huddles = False   # Don't use this here yet; 
                                 # NEED TO FIRST FIX THE HUDDLES THEN RUN THIS
                                 # See cell 3 below which does this after
cohort.fps = 24
cohort.preprocess_feature_tracks()

#
print ("DONE...")

100%|██████████████████████████████████████| 104/104 [00:00<00:00, 89865.60it/s]


DONE...


In [17]:
#########################################################
############## PREPROCESS HUDDLES #######################
#########################################################

# NOTE The hybrid video processing seems to break when using parallel flag

# 
cohort.n_cores = 30 
cohort.parallel = False 
cohort.fix_track_flag = True 
cohort.interpolate_flag = True 
cohort.overwrite_flag = False 
cohort.no_huddle_features = True    # THIS ISN'T USED HERE!
cohort.exclude_huddles = False
cohort.fps = 24
cohort.preprocess_huddle_tracks()

# 
print ("DONE...")



# of files:  117


100%|██████████████████████████████████████| 104/104 [00:00<00:00, 50522.08it/s]

light_switch time:  9:36 13824 ['day', 'night']
light_switch time:  9:36 13824 ['day', 'night']
light_switch time:  9:22 13488 ['day', 'night']
light_switch time:  9:22 13488 ['day', 'night']
light_switch time:  8:26 12144 ['day', 'night']
light_switch time:  8:26 12144 ['day', 'night']
light_switch time:  18:20 26400 ['night', 'day']
light_switch time:  18:20 26400 ['night', 'day']
light_switch time:  8:52 12768 ['day', 'night']
light_switch time:  8:52 12768 ['day', 'night']
light_switch time:  18:55 27240 ['night', 'day']
light_switch time:  18:55 27240 ['night', 'day']
light_switch time:  7:19 10536 ['day', 'night']
light_switch time:  7:19 10536 ['day', 'night']
light_switch time:  17:11 24744 ['night', 'day']
light_switch time:  17:11 24744 ['night', 'day']
light_switch time:  5:08 7392 ['day', 'night']
light_switch time:  5:08 7392 ['day', 'night']
light_switch time:  15:09 21816 ['night', 'day']
light_switch time:  15:09 21816 ['night', 'day']
light_switch time:  2:59 4296 ['da

In [21]:
#######################################################
########### REMOVE HUDDLES FROM FEATURE TRACKS ########
#######################################################

#
cohort.huddle_min_distance = 200    # min distance from huddle in pixels for feature to be kept

# this uses the _spine.npy data for tracks; and _huddle_spine_fixed_interpolated.npy for huddles
# if we want to use something else, we can also code it
cohort.parallel = True
cohort.n_cores = 30
cohort.remove_huddles_from_feature_tracks()

#


# file pairs found:  991  (if less  than above, please check missing)


  0%|          | 0/991 [00:00<?, ?it/s]